Server in esecuzione sulla Pynq.

Cattura i frame della fotocamera collegata alla pynq, li trasforma in formato YUV 4:2:0 con dimensioni 270x240 pixel, cifra ogni frame e lo invia ad un client remoto.

In [ ]:
import cv2
import numpy as np
import time
import socket
from pynq import Overlay
import pynq.lib.dma


##################### Configuration ########################

UDP_IP = "192.168.2.1"
UDP_PORT = 5555

COMPRESSED_WIDTH = 240
COMPRESSED_HEIGHT = 180

data_size = 7200

############################################################


# Caricamento dell'overlay
overlay = Overlay('/home/xilinx/pynq/overlays/AES_Encryption/design_1.bit')

# Caricamento del DMA
dma = overlay.axi_dma_0

# Alloca memoria per l'input e l'output -> 2 array da 7200 byte
in_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)
out_buffer = pynq.allocate(shape=(data_size,), dtype=np.uint8)


# Apertura della socket UDP
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)


# Inizializza la telecamera
cap = cv2.VideoCapture(2)

startTime = time.time()

# INVIO DI 50 FRAME CIFRATI
while i in range(50):

    # Leggo un frame
    _, actualFrame = cap.read()

    ############### Resize, cifratura e invio dell'immagine   ####################

    # resize
    frame = cv2.resize(actualFrame, (COMPRESSED_WIDTH, COMPRESSED_HEIGHT))

    # RGB -> YUV
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV_I420)

    # Ottieni i dati del frame come bytes
    frame_data = frame.tobytes()
    # Crea un bytearray dalla data dell'immagine
    plaintext = bytearray(frame_data)

    encrypted_img_bytes = b""

    for i in range(0, len(plaintext), data_size):
        chunk = plaintext[i:i+data_size]

        # Copia dei dati nel buffer
        np.copyto(in_buffer, chunk)

        # Avviare il trasferimento del DMA ed aspettare i risultati
        dma.sendchannel.transfer(in_buffer)
        dma.recvchannel.transfer(out_buffer)
        dma.sendchannel.wait()
        dma.recvchannel.wait()

        encrypted_img_bytes = encrypted_img_bytes + bytes(bytearray(out_buffer))

    # Invio del cifrato in un pacchetto UDP 
    sock.sendto(encrypted_img_bytes, (UDP_IP, UDP_PORT))

    ##########################################################################################

endTime = time.time()
print('Time: ', endTime - startTime, "s")
print("fps =", 50 / (endTime - startTime))

# Rilascia la telecamera e chiudi la finestra
cap.release()
cv2.destroyAllWindows()


